In [1]:
# imports
import json
import requests
from getpass import getpass
import sys
import time
import argparse
import os
import pandas as pd
import warnings
import threading
import re
import datetime
warnings.filterwarnings("ignore")


In [2]:
path = "/home/a.jajodia.229/mangrove/mount/Machine Learning Datasets/NAIP Data" # Fill a valid path to save the downloaded files
maxthreads = 5 # Threads count for downloads
sema = threading.Semaphore(value=maxthreads)
label = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") # Customized label using date time
threads = []

In [3]:
# Send http request
def sendRequest(url, data, apiKey = None, exitIfNoResponse = True):
    """
    Send a request to an M2M endpoint and returns the parsed JSON response.

    Parameters:
    endpoint_url (str): The URL of the M2M endpoint
    payload (dict): The payload to be sent with the request

    Returns:
    dict: Parsed JSON response
    """  
    
    json_data = json.dumps(data)
    
    if apiKey == None:
        response = requests.post(url, json_data)
    else:
        headers = {'X-Auth-Token': apiKey}              
        response = requests.post(url, json_data, headers = headers)  
    
    try:
      httpStatusCode = response.status_code 
      if response == None:
          print("No output from service")
          if exitIfNoResponse: sys.exit()
          else: return False
      output = json.loads(response.text)
      if output['errorCode'] != None:
          print(output['errorCode'], "- ", output['errorMessage'])
          if exitIfNoResponse: sys.exit()
          else: return False
      if  httpStatusCode == 404:
          print("404 Not Found")
          if exitIfNoResponse: sys.exit()
          else: return False
      elif httpStatusCode == 401: 
          print("401 Unauthorized")
          if exitIfNoResponse: sys.exit()
          else: return False
      elif httpStatusCode == 400:
          print("Error Code", httpStatusCode)
          if exitIfNoResponse: sys.exit()
          else: return False
    except Exception as e: 
          response.close()
          print(e)
          if exitIfNoResponse: sys.exit()
          else: return False
    response.close()
    
    return output['data']

In [4]:
def prompt_ERS_login(serviceURL):
    print("Logging in...\n")

    p = ['Enter EROS Registration System (ERS) Username: ', 'Enter ERS Account Token: ']

    # Use requests.post() to make the login request
    response = requests.post(f"{serviceUrl}login-token", json={'username': getpass(prompt=p[0]), 'token': getpass(prompt=p[1])})

    # Check for successful response
    if response.status_code == 200:  
        apiKey = response.json()['data']
        print('\nLogin Successful, API Key Received!')
        headers = {'X-Auth-Token': apiKey}
        return apiKey
    else:
        print("\nLogin was unsuccessful, please try again or create an account at: https://ers.cr.usgs.gov/register.")

In [5]:
username = "AJajodia"
token = "0m0rcKmJeF2cmRF!qXMl_beHTJ1QB2eITkJyozPuNGwX7hR86EtCCeoelC1p3Iou"

serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"
login_payload = {'username' : username, 'token' : token}
    
apiKey = sendRequest(serviceUrl + "login-token", login_payload)
    
print("API Key: " + apiKey + "\n")

API Key: eyJjaWQiOjI3NjAzODI2LCJzIjoiMTc1MTkyMjgwMSIsInIiOjYxNywicCI6WyJ1c2VyIiwiZG93bmxvYWQiLCJvcmRlciJdfQ==



In [6]:
def create_search_payload(datasetName, metadataFilter, acquisitionFilter):
    # create search payload from input filters
    search_payload = {
        'maxResults': 10,
        'datasetName' : datasetName,
        'sceneFilter' : {
            'metadataFilter': metadataFilter,
            'acquisitionFilter' : acquisitionFilter,
            'cloudCoverFilter' : {'min' : 0, 'max' : 50}
        }
    }
    return search_payload

In [7]:
sendRequest(serviceUrl + 'dataset-filters', {'datasetName': 'NAIP'}, apiKey)

[{'id': '5e83a340a3131f87',
  'legacyFieldId': 25508,
  'dictionaryLink': 'https://www.usgs.gov/centers/eros/science/national-agriculture-imagery-program-naip-data-dictionary#naip_entity_id',
  'fieldConfig': {'type': 'Text',
   'filters': [{'type': 'StringToUpper', 'options': {}},
    {'type': 'Application\\Filter\\Like', 'options': {}}],
   'options': {'size': '30'},
   'validators': [],
   'displayListId': ''},
  'fieldLabel': 'Entity ID',
  'searchSql': 'naip_entity_id like ?'},
 {'id': '63487da1fffd134b',
  'legacyFieldId': None,
  'dictionaryLink': 'https://www.usgs.gov/centers/eros/science/national-agriculture-imagery-program-naip-data-dictionary#project_name',
  'fieldConfig': {'type': 'Text',
   'filters': [],
   'options': {'size': '100'},
   'validators': [],
   'displayListId': ''},
  'fieldLabel': 'Project',
  'searchSql': 'PROJECT_NAME like ?'},
 {'id': '5e83a340bf1e1ee9',
  'legacyFieldId': 25510,
  'dictionaryLink': 'https://www.usgs.gov/centers/eros/science/national-ag

In [8]:
payload = {'datasetName' : 'NAIP',
           'sceneFilter': {'metadataFilter': {'filterType': 'and', 'childFilters': [{'filterType': 'value',
                                'filterId': '62a87101c2b8abd4',
                                'value': '1.00'}, {'filterType': 'value',
                                'filterId': '5e83a340bf1e1ee9',
                                'value': '%FL%'}]}}, 'maxResults': 50,
            'spatialFilter' : {'filterType' : "mbr",
                      'lowerLeft' : {'latitude' : 25.3, 'longitude' : -80.9},
                      'upperRight' : { 'latitude' : 25.5, 'longitude' : -81.1}}}

scenes = sendRequest(serviceUrl + 'scene-search', payload, apiKey)

In [9]:
sceneIds = []
for result in scenes['results']:
    # Add this scene to the list I would like to download
    sceneIds.append(result['entityId'])

In [10]:
sceneIds

['2545639',
 '2545641',
 '2545643',
 '2545644',
 '2545645',
 '2545646',
 '2545647',
 '2545648',
 '2545649',
 '2545650',
 '2545651',
 '2545652',
 '2545653',
 '2545654',
 '2545663',
 '2545665',
 '2545667',
 '2545668',
 '2545669',
 '2545670',
 '2545671',
 '2545684',
 '2545686',
 '2545687',
 '2545690',
 '2545691',
 '2545695',
 '2545696',
 '2545697',
 '2545698',
 '2545699',
 '2545700',
 '2545701',
 '2545702',
 '2545703',
 '2545704',
 '2545705',
 '2545706',
 '2545707',
 '2545720',
 '2545722',
 '2545724',
 '2545725',
 '2545726',
 '2545727',
 '2545739',
 '2545741',
 '2545743',
 '2545744',
 '2545745']

In [11]:
def downloadFile(url):
    sema.acquire()
    global path
    try:        
        response = requests.get(url, stream=True)
        disposition = response.headers['content-disposition']
        filename = re.findall("filename=(.+)", disposition)[0].strip("\"")
        print(f"Downloading {filename} ...\n")
        if path != "" and path[-1] != "/":
            filename = "/" + filename
        open(path + filename, 'wb').write(response.content)
        print(f"Downloaded {filename}\n")
        sema.release()
    except Exception as e:
        print(f"Failed to download from {url}. {e}. Will try to re-download.")
        sema.release()
        runDownload(threads, url)
    
def runDownload(threads, url):
    thread = threading.Thread(target=downloadFile, args=(url,))
    threads.append(thread)
    thread.start()

In [12]:
payload = {'datasetName' : 'NAIP', 'entityIds' : sceneIds}
                                
downloadOptions = sendRequest(serviceUrl + "download-options", payload, apiKey)

In [13]:
downloads = []
for product in downloadOptions:
        # Make sure the product is available for this scene
        if product['available'] == True and product['productName'] == 'Full Resolution':
                downloads.append({'entityId' : product['entityId'],
                                'productId' : product['id']})

In [14]:
payload = {'downloads' : downloads,'label' : label}
requestResults = sendRequest(serviceUrl + "download-request", payload, apiKey)   

In [15]:
requestResults

{'availableDownloads': [{'downloadId': 831819780,
   'eulaCode': None,
   'entityId': '2545724',
   'url': 'https://dds.cr.usgs.gov/download/eyJpZCI6ODMxODE5NzgwLCJjb250YWN0SWQiOjI3NjAzODI2fQ==',
   'checksum_values': [{'id': 'cksum', 'value': 1188398705}]},
  {'downloadId': 831819781,
   'eulaCode': None,
   'entityId': '2545725',
   'url': 'https://dds.cr.usgs.gov/download/eyJpZCI6ODMxODE5NzgxLCJjb250YWN0SWQiOjI3NjAzODI2fQ==',
   'checksum_values': [{'id': 'cksum', 'value': 3743789318}]},
  {'downloadId': 831819782,
   'eulaCode': None,
   'entityId': '2545726',
   'url': 'https://dds.cr.usgs.gov/download/eyJpZCI6ODMxODE5NzgyLCJjb250YWN0SWQiOjI3NjAzODI2fQ==',
   'checksum_values': [{'id': 'cksum', 'value': 2505800369}]},
  {'downloadId': 831819783,
   'eulaCode': None,
   'entityId': '2545727',
   'url': 'https://dds.cr.usgs.gov/download/eyJpZCI6ODMxODE5NzgzLCJjb250YWN0SWQiOjI3NjAzODI2fQ==',
   'checksum_values': [{'id': 'cksum', 'value': 3784861203}]},
  {'downloadId': 831819785,
  

In [16]:
for download in requestResults['availableDownloads'] :
    downloadFile(download['url'])


Downloaded /m_2708013_ne_17_1_20171211.ZIP


Downloaded /m_2708013_nw_17_1_20171211.ZIP


Downloaded /m_2708013_se_17_1_20171211.ZIP


Downloaded /m_2708013_sw_17_1_20171211.ZIP


Downloaded /m_2708028_ne_17_1_20171211.ZIP


Downloaded /m_2708028_se_17_1_20171211.ZIP


Downloaded /m_2708029_ne_17_1_20171211.ZIP


Downloaded /m_2708029_nw_17_1_20171211.ZIP


Downloaded /m_2708029_se_17_1_20171211.ZIP


Downloaded /m_2708029_sw_17_1_20171211.ZIP


Downloaded /m_2708030_nw_17_1_20171211.ZIP


Downloaded /m_2708030_se_17_1_20171211.ZIP


Downloaded /m_2708038_se_17_1_20171211.ZIP


Downloaded /m_2708038_sw_17_1_20171211.ZIP


Downloaded /m_2708044_ne_17_1_20171211.ZIP


Downloaded /m_2708044_se_17_1_20171211.ZIP


Downloaded /m_2708052_ne_17_1_20171211.ZIP


Downloaded /m_2708052_se_17_1_20171211.ZIP


Downloaded /m_2708053_ne_17_1_20171211.ZIP


Downloaded /m_2708053_nw_17_1_20171211.ZIP


Downloaded /m_2708053_se_17_1_20171211.ZIP


Downloaded /m_2708053_sw_17_1_20171211.ZIP


Downloade